In [1]:
import cobra
from cobra import Metabolite, Reaction
import gurobipy

#These three functions are modifications of code by Linn Sandvik.
def make_reaction (df, model):
    """
    df: Takes in a dataframe with two columns; metabolite IDs in one and their coefficients (mmol/gDW) in the other. 
    model: Takes in the model that contains the metabolite objects
    Return: This function will returen a reaction with metabolite objects (from the metabolite IDs in the dataframes) with corresponding coefficients from df.
    Essentially pairs the coefficients to the metabolite objects already in the model. Important so that both coefficients and other potential attributes already
    in the model are all paired to the metabolite."""

    metabolite_to_coef = {} #Dictionary, maps metabolite object to coef. Float.
    for list in df.values.tolist():
        metabolite_to_coef[model.metabolites.get_by_id(list[1])] = float(list[0])


    reaction = Reaction ()
    reaction.add_metabolites(metabolite_to_coef)
    return reaction

def modify_reaction (reaction, reaction_id, model):

    """
    Reaction: The reaction object
    Reaction_id: The ID of the reaction where the attributes are gathered from.
    Model: The model that contains the reaction object with reaction ID.
    Return: Returns object with the same id, name, substystem, lower- and upper bound as reaction object from model with id = reaction_id.
    Returns a rection variable that has the same attributes as the corresponding reaction from the model. Creates a new reaction object that 
    is connected to the attributes of the existing reaction in the model.
    """

    reaction.id = model.reactions.get_by_id(reaction_id).id + "new"
    reaction.name = model.reactions.get_by_id(reaction_id).name
    reaction.subsystem = model.reactions.get_by_id(reaction_id).subsystem
    reaction.lower_bound = model.reactions.get_by_id(reaction_id).lower_bound
    reaction.upper_bound = model.reactions.get_by_id(reaction_id).upper_bound
    return reaction


def new_synthesis_reaction(df, rxn_id, model):
    """
    :param df: DataFrame with two columns: Coefficient (mmol/gDW) and metabolite ID.
    :param rxn_id: Reaction ID of the original synthesis Reaction object.
    :param model: The metabolic model in use.
    :return: New synthesis Reaction object with the metabolites and coefficients from df.
    """
    synthesis_reaction = make_reaction(df, model)
    synthesis_reaction = modify_reaction(synthesis_reaction, rxn_id, model)

    return synthesis_reaction


In [2]:
import pandas as pd 
import os

os.getcwd()
os.chdir("C:/Users/sofie/OneDrive/Documents/Master - Data/")
iBsu1147 = cobra.io.read_sbml_model("iBsu1147.xml")
iBsu1147og = cobra.io.read_sbml_model("iBsu1147.xml")

#The name and ID is mixed up. This is changed here so that metabolite.name returns the name, and metabolite id. returns ID.

for metabolite in iBsu1147.metabolites:
    name = metabolite.id
    id = metabolite.name

    metabolite.name = name
    metabolite.id = id

for metabolite in iBsu1147og.metabolites:
    name = metabolite.id
    id = metabolite.name

    metabolite.name = name
    metabolite.id = id

Set parameter Username
Academic license - for non-commercial use only - expires 2024-04-25


In [3]:
#COFACTORS AND IONS:
#Creating the cofactor synthesis reaction and creating the "dummy" metabolite; cofactor.
os.chdir("C:/Users/sofie/OneDrive/Documents/Master - Data/")

cofactor = Metabolite( id="Cofactor[c]", formula= None, name="cofactor", charge= None, compartment= "c")
iBsu1147.add_metabolites(cofactor)


cofactors_ions_df = pd.read_excel("Macromolecule-synthesis.xlsx","Cofactor") 
cofac_synthesis= make_reaction(cofactors_ions_df, iBsu1147)
cofac_synthesis.name = "cofac_synthesis"
cofac_synthesis.id = "cofactor"
cofac_synthesis.lower_bound = -1000.0
cofac_synthesis.upper_bound = 1000

print(cofac_synthesis)


cofactor: 0.406841373641698 C00003[c] + 0.005285365155927 C00005[c] + 0.0235128411879642 C00006[c] + 0.00284963130743174 C00010[c] + 0.117296253333691 C00020[c] + 0.00494595852149485 C00035[c] + 0.0109034381311329 C00044[c] + 0.0257457795723863 C00055[c] + 0.0136320888368966 C00063[c] + 0.0666085520073099 C00076[c] + 0.00652911183605009 C00112[c] + 0.0132614210650826 C00144[c] + 0.00461840878988398 C00234[c] + 14.6838028159594 C00238[c] + 2.11548582540145 C00305[c] + 0.00334427181834891 C00828[c] + 0.0716549927561037 C14819[c] <=> Cofactor[c]


In [4]:
#Creating  the DNA synthesis reaction:
DNA_df = pd.read_excel("Macromolecule-synthesis.xlsx","DNA")
DNA_synthesis = make_reaction(DNA_df, iBsu1147)
modify_reaction(DNA_synthesis, "rxn05294", iBsu1147)
print(DNA_synthesis)

rxn05294new: 0.925419454369164 C00131[c] + 0.700555089212493 C00286[c] + 0.699717605543381 C00458[c] + 0.921964834234075 C00459[c] <=> 3.24765698335911 C00013[c] + C00039[c]


In [5]:
#Creating  the RNA synthesis reaction:
#print(iBsu1147.reactions.get_by_id("rxn05295"))
RNA_df = pd.read_excel("Macromolecule-synthesis.xlsx","RNA")
RNA_synthesis = make_reaction(RNA_df, iBsu1147)

RNA_synthesis = modify_reaction(RNA_synthesis, "rxn05295", iBsu1147)

print(RNA_synthesis)
#funker.

rxn05295new: 0.810318111715803 C00002[c] + 0.998544094063492 C00044[c] + 0.615467416022917 C00063[c] + 0.665731114102355 C00075[c] --> 3.09006073590457 C00013[c] + cpd11462[c]


In [6]:
#Creating  the lipid synthesis reaction:
Lipid_df = pd.read_excel("Macromolecule-synthesis.xlsx","Lipid")
Lipid_synthesis = make_reaction(Lipid_df, iBsu1147)

Lipid_synthesis = modify_reaction(Lipid_synthesis, "rxn10201", iBsu1147)

print(Lipid_synthesis)
print(iBsu1147.reactions.get_by_id("rxn10201"))

rxn10201new: 0.0182341974255657 cpd15529[c] + 0.0735693637318944 cpd15531[c] + 0.0249314494535681 cpd15533[c] + 0.00572228475376097 cpd15536[c] + 0.0231742993712587 cpd15538[c] + 0.00787805688331436 cpd15540[c] + 0.0539093645252547 cpd15695[c] + 0.126715623078546 cpd15696[c] + 0.00871948075130027 cpd15697[c] + 0.146496112576544 cpd15698[c] + 0.248310408774362 cpd15699[c] + 0.0342393244610017 cpd15700[c] + 0.00874458289533326 cpd15707[c] + 0.00210355966996477 cpd15708[c] + 0.00304137577103737 cpd15709[c] + 0.00649442670421581 cpd15710[c] + 0.0152621035720594 cpd15711[c] + 0.00100609393284234 cpd15712[c] + 0.0171698665185669 cpd15713[c] + 0.0290883645054317 cpd15714[c] + 0.00406955959062874 cpd15715[c] + 0.0170092127967557 cpd15722[c] + 0.0399826950157504 cpd15723[c] + 0.0027371377853575 cpd15724[c] + 0.046077495586961 cpd15725[c] + 0.0780777088002198 cpd15726[c] + 0.0107838810765736 cpd15727[c] + 0.014529120966296 cpd15728[c] + 0.00353136962256138 cpd15729[c] + 0.00500938386322398 cpd15

In [7]:
#Creating  the lipoteichoic acid synthesis reaction:
Lipo_df = pd.read_excel("Macromolecule-synthesis.xlsx", "Lipoteichoic_acid")
Lipo_synthesis = make_reaction(Lipo_df, iBsu1147)
Lipo_synthesis = modify_reaction(Lipo_synthesis, "rxn10200", iBsu1147)


print(Lipo_synthesis)
print(iBsu1147.reactions.get_by_id("rxn10200"))

rxn10200new: 0.00488084303736876 cpd15746[c] + 0.00112542250060168 cpd15747[c] + 0.00176637167875183 cpd15748[c] + 0.00369824666641567 cpd15749[c] + 0.00869343744448727 cpd15750[c] + 0.000538136516553297 cpd15751[c] + 0.00938453914048955 cpd15752[c] + 0.0158983481591235 cpd15753[c] + 0.0022709060083442 cpd15754[c] + 0.00227592126012941 cpd15755[c] + 0.0005216864906978 cpd15756[c] + 0.000828218679809985 cpd15757[c] + 0.00172925881554125 cpd15758[c] + 0.00406536309709322 cpd15759[c] + 0.000249558928832172 cpd15760[c] + 0.00436326905313484 cpd15761[c] + 0.00739248113140316 cpd15762[c] + 0.00105922117703687 cpd15763[c] + 0.00203819832551034 cpd15764[c] + 0.00046691994120328 cpd15765[c] + 0.000742357569247148 cpd15766[c] + 0.00154971280163065 cpd15767[c] + 0.00364207584642129 cpd15768[c] + 0.000223379314513363 cpd15769[c] + 0.0039068811406805 cpd15770[c] + 0.0066191293061234 cpd15771[c] + 0.000948584722655084 cpd15772[c] + 0.00646064734971069 cpd15773[c] + 0.00148451452842289 cpd15774[c] + 

In [8]:
#Creating the Cell wall synthesis reaction: 

Cellwall_df = pd.read_excel("Macromolecule-synthesis.xlsx","Cell_wall")
Cellwall_synthesis = make_reaction(Cellwall_df, iBsu1147)
Cellwall = modify_reaction(Cellwall_synthesis, "rxn10198", iBsu1147)

print(Cellwall)

rxn10198new: 0.0145386839210681 cpd11459[c] + 0.454208539051299 cpd15665[c] + 0.0160426857060061 cpd15667[c] + 0.0112298799942043 cpd15668[c] + 0.0081015562815331 cpd15669[c] <=> cpd15664[c] + 0.489582661033042 cpd15666[c]


In [9]:
#Importing the synthesis reaction for proteins containing the measured amino acid concentrations:
PGlucose_df = pd.read_excel("Macromolecule-synthesis.xlsx", "Proteins_Gluc")
PGlucose_synthesis = make_reaction(PGlucose_df, iBsu1147)
PGlucose_synthesis = modify_reaction(PGlucose_synthesis, "rxn05296", iBsu1147)

In [10]:
#Creating the new 
Datafile = pd.read_excel("BOF_ny.xlsx", "Glucose")

BOF_df = pd.DataFrame(Datafile)
                      
BOFrx = make_reaction(BOF_df, iBsu1147)

BOFnew = modify_reaction(BOFrx, "bio00006", iBsu1147)

print (BOFnew)
print(iBsu1147.reactions.get_by_id("bio00006"))

bio00006new: 105.0 C00001[c] + 105.003 C00002[c] + 0.572154094567705 C00017[c] + 0.023458553031822 C00039[c] + 0.000273109 C00229[c] + 0.0446336317745074 Cofactor[c] + 0.0697874204115724 cpd11462[c] + 0.2242 cpd15664[c] + 0.0304 cpd15670[c] + 0.0311157979687115 cpd15800[c] --> 104.997 C00008[c] + 104.987 C00009[c] + 0.000273109 C03688[c]
bio00006: 105.0 C00001[c] + 105.003 C00002[c] + 0.01822 C00003[c] + 0.0002367 C00005[c] + 0.001053 C00006[c] + 0.000127618 C00010[c] + 0.0008548 C00013[c] + 0.5284 C00017[c] + 0.005253 C00020[c] + 0.0002215 C00035[c] + 0.026 C00039[c] + 0.0004883 C00044[c] + 0.001153 C00055[c] + 0.0006105 C00063[c] + 0.002983 C00076[c] + 0.0002924 C00112[c] + 0.0005939 C00144[c] + 0.000273109 C00229[c] + 0.000206831 C00234[c] + 0.6576 C00238[c] + 0.09474 C00305[c] + 0.00014977 C00828[c] + 0.003209 C14819[c] + 0.0655 cpd11462[c] + 0.2242 cpd15664[c] + 0.0304 cpd15670[c] + 0.076 cpd15800[c] --> 104.997 C00008[c] + 104.987 C00009[c] + 0.000273109 C03688[c]


In [1]:
#Removing old BOF synthesis reactions:
SynthesisIDs = ["rxn05294", "rxn05295", "rxn05296", "rxn10198", "rxn10200", "rxn10201", "bio00006"]
for IDs in SynthesisIDs:
    iBsu1147.remove_reactions([IDs])

print(len(iBsu1147.reactions))

#Adding new reactions:
synthesis_reactions = (cofac_synthesis, DNA_synthesis, RNA_synthesis, Lipid_synthesis, Lipo_synthesis, Cellwall, PGlucose_synthesis, BOFnew)

for reactions in synthesis_reactions:
    iBsu1147.add_reactions([reactions])

print(len(iBsu1147.reactions))

#printing the length of the model when removing the original reactions and adding the new ones to check that everything went as it should.

NameError: name 'iBsu1147' is not defined

In [ ]:
# importing df that contains the components of the minimal medium used experimentally
os.getcwd()
os.chdir("C:/Users/sofie/OneDrive/Documents/Master - Data/")

experimental_medium_df = pd.read_excel("Medium.xlsx", "Glucose")

Exchange_reactions_medium = experimental_medium_df["Exchange reaction ID"]

exchange_reactions = Exchange_reactions_medium.values.tolist()

#The bacteria in the batch fermentation experiments were incubated in minimal media with their respecitive C-source.
#In order to make sure the model predictions are based on what was available to the bacteria during the experiments,
#the flux ranges for the exchange reactions for components in the media must be set to -1000 and 1000. 
#That means that the components that were in the cultivation media, is the same as what is "available" in the model.

for exchange in iBsu1147.exchanges:
    if exchange.id in exchange_reactions:  # if the component from the exchange reaction was
        # available in the medium, the model should be able to take up and secrete it.
        iBsu1147.reactions.get_by_id(exchange.id).lower_bound = -1000.0
        iBsu1147.reactions.get_by_id(exchange.id).upper_bound = 1000.0
    else:
        # if the exchange reaction is not part of the medium, it can't be taken up from the media. However,
        # it is possible for the bacteria to secrete it.
        iBsu1147.reactions.get_by_id(exchange.id).lower_bound = 0.0
        iBsu1147.reactions.get_by_id(exchange.id).upper_bound = 1000.0

#changing the medium composition for the model with the original BOF:
for exchange in iBsu1147og.exchanges:
    if exchange.id in exchange_reactions: 
        iBsu1147og.reactions.get_by_id(exchange.id).lower_bound = -1000.0
        iBsu1147og.reactions.get_by_id(exchange.id).upper_bound = 1000.0
    else:
        iBsu1147og.reactions.get_by_id(exchange.id).lower_bound = 0.0
        iBsu1147og.reactions.get_by_id(exchange.id).upper_bound = 1000.0

In [13]:
#Setting the new objective function:
iBsu1147.objective = "bio00006new"
print(iBsu1147.objective)
print(iBsu1147.optimize())

New_reactions = [cofac_synthesis, DNA_synthesis, RNA_synthesis, Lipid_synthesis, Lipo_synthesis, Cellwall
                 , PGlucose_synthesis, BOFnew]

for reaction in New_reactions:
    print(reaction.id, iBsu1147.optimize().fluxes[reaction.id])

Maximize
1.0*bio00006new - 1.0*bio00006new_reverse_1501f
<Solution 0.100 at 0x22f7fc437c0>
cofactor 0.0044633631774507404
rxn05294new 0.0023458553031822003
rxn05295new 0.00697874204115724
rxn10201new 0.0031115797968711504
rxn10200new 0.00304
rxn10198new 0.022420000000000002
rxn05296new 0.057215409456770505
bio00006new 0.1


In [14]:
#Setting the experimentally measured growth rate and running the model: 
iBsu1147.reactions.get_by_id("bio00006new").upper_bound = (0.4107) #Measured growth rate (0.3567 + (2*0.054)) to give the model more flexibility.
iBsu1147.reactions.get_by_id("bio00006new").lower_bound = (0.3027)

print(iBsu1147.optimize())

SynthesisIDs = ["rxn05294new", "rxn05295new", "rxn05296new","rxn10198new", "rxn10200new", "rxn10201new", "bio00006new", "cofactor"]

for reactionID in SynthesisIDs:
    print("Flux through reaction", reactionID, ":", iBsu1147.reactions.get_by_id(reactionID).flux)

<Solution 0.411 at 0x22f7fc43670>
Flux through reaction rxn05294new : 0.009634427730169296
Flux through reaction rxn05295new : 0.028661693563032786
Flux through reaction rxn05296new : 0.23498368663895647
Flux through reaction rxn10198new : 0.09207894000000001
Flux through reaction rxn10200new : 0.01248528
Flux through reaction rxn10201new : 0.012779258225749813
Flux through reaction bio00006new : 0.4107
Flux through reaction cofactor : 0.01833103256979019


In [15]:
#Setting the experimentally measured growth rate and running the model with the original BOF: 
iBsu1147og.reactions.get_by_id("bio00006").upper_bound = (0.4107) #Measured growth rate (0.3567 + (2*0.054)) to give the model more flexibility.
iBsu1147og.reactions.get_by_id("bio00006").lower_bound = (0.3027)

iBsu1147og.optimize()
SynthesisIDsog = ["rxn05294", "rxn05295", "rxn05296","rxn10198", "rxn10200", "rxn10201", "bio00006"]

for reactionID in SynthesisIDsog:
    print("Flux through reaction", reactionID, ":", iBsu1147og.reactions.get_by_id(reactionID).flux)

Flux through reaction rxn05294 : 0.0106782
Flux through reaction rxn05295 : 0.02690085
Flux through reaction rxn05296 : 0.21701388
Flux through reaction rxn10198 : 0.09207894000000001
Flux through reaction rxn10200 : 0.01248528
Flux through reaction rxn10201 : 0.0312132
Flux through reaction bio00006 : 0.4107


In [2]:
#Setting the experimentally measured fluxes
import cobra

#This function takes in the model and a dictionary of measured fluxes and updates the fluxes for the reactions. 
def update_fluxes_in_model(model, fluxes):
    for reaction_id, flux in fluxes.items():
        if reaction_id in model.reactions:
            reaction = model.reactions.get_by_id(reaction_id)
            lower_bound, upper_bound = flux  # Extract lower and upper bounds from the tuple
            reaction.lower_bound = lower_bound
            reaction.upper_bound = upper_bound
            #print(f"Flux updated for reaction ID {reaction_id}.")
        else:
            print(f"Reaction ID {reaction_id} not found in the model.")

#Setting the experimentally measured fluxes in glucose resulted in the model not being able to provide a feasible solution. 
#That is why the reaction fluxes here are inside there """ """. The experimentally measured fluxes should not be set before running the MOMA
#analysis to find a feasible distribution of fluxes. Only the growth rate is locked for the MOMA analysis. Measured fluxes are given 
#as a reference.

"""Fluxes = {"E00096": (-3.52, 1000), "E00068": (0, 1000), "E00117": (0, 1000), "E00046": (0, 1000), "E00017": (0, 1000), "E00012":  (0, 3.6),
          "E00032": (-0.02,1000), "E00009": (-1.98, 1000), "E00002": (-5.22, 1000), "E00004": (0, 5.49), "EX_Methanol[c]": (0, 1000)}

update_fluxes_in_model(iBsu1147, Fluxes)  # Update fluxes in the model using the Fluxes dictionary

iBsu1147.optimize()


Fluxes = {"E00096": (-3.52, 1000), "E00068": (0, 1000), "E00117": (0, 1000), "E00046": (0, 1000), "E00017": (0, 1000), "E00012":  (0, 3.6),
          "E00032": (-0.02,1000), "E00009": (-1.98, 1000), "E00002": (-5.22, 1000), "E00004": (0, 5.49), "EX_Methanol[c]": (0, 1000)}

update_fluxes_in_model(iBsu1147og, Fluxes)  # Update fluxes in the model using the Fluxes dictionary

iBsu1147og.optimize()"""

'Fluxes = {"E00096": (-3.52, 1000), "E00068": (0, 1000), "E00117": (0, 1000), "E00046": (0, 1000), "E00017": (0, 1000), "E00012":  (0, 3.6),\n          "E00032": (-0.02,1000), "E00009": (-1.98, 1000), "E00002": (-5.22, 1000), "E00004": (0, 5.49), "EX_Methanol[c]": (0, 1000)}\n\nupdate_fluxes_in_model(iBsu1147, Fluxes)  # Update fluxes in the model using the Fluxes dictionary\n\niBsu1147.optimize()\n\n\nFluxes = {"E00096": (-3.52, 1000), "E00068": (0, 1000), "E00117": (0, 1000), "E00046": (0, 1000), "E00017": (0, 1000), "E00012":  (0, 3.6),\n          "E00032": (-0.02,1000), "E00009": (-1.98, 1000), "E00002": (-5.22, 1000), "E00004": (0, 5.49), "EX_Methanol[c]": (0, 1000)}\n\nupdate_fluxes_in_model(iBsu1147og, Fluxes)  # Update fluxes in the model using the Fluxes dictionary\n\niBsu1147og.optimize()'

In [17]:
import reframed 
from reframed import from_cobrapy
from reframed import FBA
from reframed import MOMA

#Performing the MOMA analysis. The growth-rate was already set. Measured fluxes are used as a reference. 
#This is the analysis on the model with then new BOF.

cbmodel = reframed.from_cobrapy(iBsu1147)

Fluxes = {"E00096": -3.512, "E00068": 0 , "E00117": 0 , "E00046": 0 , "E00017": 0 , "E00012":  3.6, 
          "E00032": -0.021, "E00009": -1.985, "E00002": -5.2, "E00004": 5.5, "EX_Methanol[c]": 0}

ReactionIDsnew = ["rxn05294new", "rxn05295new", "rxn05296new","rxn10198new", "rxn10200new", "rxn10201new", "bio00006new", "cofactor",  "E00096", "E00068", "E00017", "E00117", "E00046", "E00012", "E00008", "E00032", "E00009", "E00002", "E00004"]



modelMOMAglu = MOMA(cbmodel, reference= Fluxes, reactions=ReactionIDsnew)
print(modelMOMAglu)
for reactions in ReactionIDsnew:
    print(modelMOMAglu.show_values(reactions))

Objective: -82.01355145405978
Status: Optimal

rxn05294new   0.0071009
None
rxn05295new   0.0211247
None
rxn05296new   0.173191
None
rxn10198new   0.0678653
None
rxn10200new   0.00920208
None
rxn10201new   0.00941875
None
bio00006new   0.3027
None
cofactor      0.0135106
None
E00096       -4.86329
None

None

None

None

None
E00012        3.6
None
E00008        3.79915
None
E00032       -0.0137854
None
E00009       -2.06222
None
E00002       -6.74433
None
E00004        6.00191
None


In [41]:
#FBA with results from MOMA analysis: 
import reframed 
from reframed import from_cobrapy
from reframed import FBA

#The new suggested fluxes from the MOMA analysis are listed below. These were updated and then the model was saved with this growth rate and these fluxes.

iBsu1147.reactions.get_by_id("bio00006new").upper_bound = 0.3027
iBsu1147.reactions.get_by_id("bio00006new").lower_bound = 0.3010
 
new_Fluxesglu = {"E00096": (-4.863, 1000), "E00068": (0, 1000), "E00117": (0, 1000), "E00046": (0, 1000), "E00017": (0, 1000), "E00012":  (0, 3.6), "E00008": (0, 3.799),
          "E00032": (-0.01378, 1000), "E00009": (-2.062, 1000), "E00002": (-6.744, 1000), "E00004": (0, 6.002), "EX_Methanol[c]": (0, 1000)}

update_fluxes_in_model(iBsu1147, new_Fluxesglu)
print(iBsu1147.optimize())

cobra.io.write_sbml_model(iBsu1147, "C:/Users/sofie/OneDrive/Documents/Master - Data/ModelGlu.sbml")

"""
#Running a FBA analysis with the MOMA-fluxes: 
cbmodelglu2 = reframed.from_cobrapy(iBsu1147)
solglu2 = FBA(cbmodelglu2)
print(solglu2.show_values())

ReactionIDs = ["rxn05294new", "rxn05295new", "rxn05296new","rxn10198new", "rxn10200new", "rxn10201new", "bio00006new", "cofactor", "E00117", "E00012", "E00032", "E00009", "E00002", "E00004"]

for reactionID in ReactionIDs:
    flux = solglu2.values.get(reactionID)
    print(reactionID, round(flux,6))"""
    
"""SynthesisIDs = ["rxn05294new", "rxn05295new", "rxn05296new","rxn10198new", "rxn10200new", "rxn10201new", "bio00006new", "cofactor"]

for reactionID in SynthesisIDs:
    print("Flux through reaction", reactionID, ":", iBsu1147.reactions.get_by_id(reactionID).flux)"""

<Solution 0.302 at 0x22f0dfafbb0>


'SynthesisIDs = ["rxn05294new", "rxn05295new", "rxn05296new","rxn10198new", "rxn10200new", "rxn10201new", "bio00006new", "cofactor"]\n\nfor reactionID in SynthesisIDs:\n    print("Flux through reaction", reactionID, ":", iBsu1147.reactions.get_by_id(reactionID).flux)'

In [36]:
#MOMA original model BOF:
import reframed
from reframed import from_cobrapy
from reframed import FBA
from reframed import MOMA


#Performing the MOMA analysis. The growth-rate was already set. Measured fluxes are used as a reference. 
#This is the analysis on the model with then original BOF.

cbmodelog = reframed.from_cobrapy(iBsu1147og)

Fluxesog = {"E00096": -3.516, "E00068": 0 , "E00117": 0 , "E00046": 0 , "E00017": 0 , "E00012":  3.6, 
          "E00032": -0.021, "E00009": -1.985, "E00002": -5.2, "E00004": 5.5, "EX_Methanol[c]": 0}

ReactionIDs = ["rxn05294", "rxn05295", "rxn05296","rxn10198", "rxn10200", "rxn10201", "bio00006", "E00096", "E00068", "E00017", "E00117", "E00046", "E00012", "E00008", "E00032", "E00009", "E00002", "E00004"]

modelMOMAogglu = MOMA(cbmodelog, reference= Fluxesog, reactions=ReactionIDs)
print(modelMOMAogglu)
for reactions in ReactionIDs:
    print(modelMOMAogglu.show_values(reactions))

#Hvordan påvirkes modellen av de nye biomassefunksjonene? kommer vi nærmere eksperimentelt målte verdier? Er det en effekt på hvor nær vi kommer eksperimentelle verdier?

Objective: -81.50254383962601
Status: Optimal

rxn05294      0.0078702
None
rxn05295      0.0198269
None
rxn05296      0.159947
None
rxn10198      0.0678653
None
rxn10200      0.00920208
None
rxn10201      0.0230052
None
bio00006      0.3027
None
E00096       -4.94576
None

None
E00017        1.16269e-09
None

None

None
E00012        3.6
None
E00008        3.51617
None
E00032       -0.0164425
None
E00009       -2.0667
None
E00002       -6.83401
None
E00004        6.03105
None


In [40]:
#The new suggested fluxes from the MOMA analysis with ORIGINAL BOF are:
#The new suggested fluxes from the MOMA analysis are listed below. These were updated and then the model was saved with this growth rate and these fluxes.

iBsu1147og.reactions.get_by_id("bio00006").upper_bound =  0.3027
iBsu1147og.reactions.get_by_id("bio00006").lower_bound = 0.3010
 
 
new_Fluxesog = {"E00096": (-4.945, 1000), "E00068": (0, 1000), "E00117": (0, 1000), "E00046": (0, 1000), "E00017": (0, 1000), "E00012":  (0, 3.6), "E00008": (0, 3.516),
          "E00032": (-0.016, 1000), "E00009": (-2.07, 1000), "E00002": (-6.83, 1000), "E00004": (0, 6.03), "EX_Methanol[c]": (0, 1000)}

update_fluxes_in_model(iBsu1147og, new_Fluxesog)
iBsu1147og.optimize()

cobra.io.write_sbml_model(iBsu1147og, "C:/Users/sofie/OneDrive/Documents/Master - Data/OGModelGlu.sbml")

In [ ]:
"""
#Running a FBA analysis with the MOMA-fluxes: 
cbmodelgluog2 = reframed.from_cobrapy(iBsu1147og)
solgluog2 = FBA(cbmodelgluog2)
#print(solcb.show_values())

SynthesisIDs = ["rxn05294", "rxn05295", "rxn05296","rxn10198", "rxn10200", "rxn10201", "bio00006",  "E00096", "E00068", "E00017", "E00117", "E00046", "E00012", "E00008", "E00032", "E00009", "E00002", "E00004"]

for reactionID in SynthesisIDs:
    flux = solgluog2.values.get(reactionID)
    print(reactionID, round(flux,6))"""

In [ ]:
#FVA:
from cobra import flux_analysis

#Update model with MOMA-results:
iBsu1147.reactions.get_by_id("bio00006new").upper_bound = 0.3027
iBsu1147.reactions.get_by_id("bio00006new").lower_bound = 0.3000
 
new_Fluxesglu = {"E00096": (-3.54, 1000), "E00068": (0, 1000), "E00117": (0, 1000), "E00046": (0, 1000), "E00017": (0, 1000), "E00012":  (0, 3.6),
          "E00032": (-0.0099, 1000), "E00009": (-1.98, 1000), "E00002": (-5.25, 1000), "E00004": (0, 5.52), "EX_Methanol[c]": (0, 1000)}


update_fluxes_in_model(iBsu1147, new_Fluxesglu)
iBsu1147.optimize()

FVAglu =flux_analysis.flux_variability_analysis(iBsu1147, loopless=True, fraction_of_optimum=0.95)

In [ ]:
print(FVAglu[1:30])

In [ ]:
#FVA old model
iBsu1147og.reactions.get_by_id("bio00006").upper_bound = 0.221
iBsu1147og.reactions.get_by_id("bio00006").lower_bound = 0.0
 
new_Fluxesog = {"E00096": (-3.54, 1000), "E00068": (0, 1000), "E00117": (0, 1000), "E00046": (0, 1000), "E00017": (0, 1000), "E00012":  (0, 3.6),
          "E00032": (-0.012, 1000), "E00009": (-1.98, 1000), "E00002": (-5.25, 1000), "E00004": (0, 5.52), "EX_Methanol[c]": (0, 1000)}

update_fluxes_in_model(iBsu1147og, new_Fluxesog)

iBsu1147og.optimize()

FVAgluog = flux_analysis.flux_variability_analysis(iBsu1147og, loopless= True, fraction_of_optimum= 0.95)


In [ ]:
#Checking to see if i can remove the biomass synthesis reactions so that i can make a plot with the FVA results.
Reactions_remove = ["rxn05294new", "rxn05295new", "rxn05296new","rxn10198new", "rxn10200new", "rxn10201new", "bio00006new", "cofactor"]

#print(FVAglu.columns)


filtered_Glu = pd.DataFrame(columns=FVAglu.columns)
removed_df = pd.DataFrame(columns=FVAglu.columns)

#I want to loop trough the reactions that are indexed in the FVA dataframe and if that idex is one of the reactions in Reactions_remove list, then this reaction
#will be reomved from the filtered dataframe. I also made another dataframe for the FVA results of the removed reactions, because it is interesting to get to see
#those results as well.
for reaction_id in FVAglu.index:
    # Check if the reaction ID is in the Reactions_remove list
    if reaction_id in Reactions_remove:
        # Append the reaction to the removed_df DataFrame
        removed_df.loc[reaction_id] = FVAglu.loc[reaction_id]
    else:
        # Append the reaction to the filtered_df DataFrame
        filtered_Glu.loc[reaction_id] = FVAglu.loc[reaction_id]

# Print the filtered DataFrame
print(filtered_Glu)
print(len(filtered_Glu))

In [ ]:
#Checking to see if i can remove the biomass synthesis reactions so that i can make a plot with the FVA results.
Reactions_remove = ["rxn05294", "rxn05295", "rxn05296","rxn10198", "rxn10200", "rxn10201", "bio00006"]

print(FVAgluog.columns)


filtered_GluOG = pd.DataFrame(columns=FVAgluog.columns)
removed_dfOG = pd.DataFrame(columns=FVAgluog.columns)

#I want to loop trough the reactions that are indexed in the FVA dataframe and if that idex is one of the reactions in Reactions_remove list, then this reaction
#will be reomved from the filtered dataframe. I also made another dataframe for the FVA results of the removed reactions, because it is interesting to get to see
#those results as well.
for reaction_id in FVAgluog.index:
    # Check if the reaction ID is in the Reactions_remove list
    if reaction_id in Reactions_remove:
        # Append the reaction to the removed_df DataFrame
        removed_dfOG.loc[reaction_id] = FVAgluog.loc[reaction_id]
    else:
        # Append the reaction to the filtered_df DataFrame
        filtered_GluOG.loc[reaction_id] = FVAgluog.loc[reaction_id]

# Print the filtered DataFrame
print(filtered_GluOG[0:30])
print(len(filtered_GluOG))

In [ ]:
print(filtered_GluOG)

In [ ]:


import matplotlib.pyplot as plt
import numpy as np

FVAglu_mid = []
for index, row in filtered_Glu.iterrows():
    mid_value = (row['minimum'] + row['maximum']) / 2
    FVAglu_mid.append(mid_value)

FVAglu_range = []
for index, row in filtered_Glu.iterrows():
    range_value = abs(row['maximum'] - row['minimum'])
    FVAglu_range.append(range_value)

FVAgluog_mid = []
for index, row in filtered_GluOG.iterrows():
    mid_value = (row['minimum'] + row['maximum']) / 2
    FVAgluog_mid.append(mid_value)

FVAgluog_range =[]
for index, row in filtered_GluOG.iterrows():
    range_valueog = abs(row["maximum"] - row["minimum"])
    FVAgluog_range.append(range_valueog)

#range_valuesGlu = [x[1] for x in FVAglu_range]
#range_valuesGluOG = [x[1] for x in FVAgluog_range]

print(FVAglu_range)
print(len(FVAglu_range))
print(FVAgluog_range)
print(len(FVAgluog_range))


In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Create a figure and axes for range values
fig, (ax1, ax2) = plt.subplots(nrows=2, ncols=1, figsize=(10, 10), sharex=True)

# Create a list of reaction numbers
reaction_numbers = range(1, len(filtered_GluOG) + 1)

# Plot the range values for model 1
ax1.scatter(reaction_numbers, FVAglu_range, color= "#1f78b4", label='New BOF', alpha= 0.6)

# Plot the range values for model 2
ax1.scatter(reaction_numbers, FVAgluog_range, color="grey", label='Original BOF', alpha= 0.4)


# Set labels for the range subplot
ax1.set_ylim(0.1,8)
ax1.set_ylabel('Range')
ax1.grid(axis='y', linestyle='--', alpha=0.5)
ax1.spines['top'].set_visible(False)
ax1.spines['right'].set_visible(False)
ax1.text(0.05, 0.95, 'A', transform=ax1.transAxes, fontsize=16, fontweight='bold')

# Plot the mid-values for model 1
ax2.scatter(reaction_numbers, FVAglu_mid, s=50, c="#1f78b4", marker='o', label='New BOF', alpha=0.6)

# Plot the mid-values for model 2
ax2.scatter(reaction_numbers, FVAgluog_mid, s=50, c="grey", marker='o', label='Original BOF', alpha=0.4)

# Set y-axis scale to logarithmic for mid-value subplot
ax2.set_yscale('log')

# Set labels for the mid-value subplot
ax2.set_xlabel('Reaction Number')
ax2.set_ylim(0.0000001, )
ax2.set_ylabel('Log(Mid-Value)')
ax2.grid(axis='y', linestyle='--', alpha=0.5)
ax2.spines['top'].set_visible(False)
ax2.spines['right'].set_visible(False)
ax2.text(0.05, 0.95, 'B', transform=ax2.transAxes, fontsize=16, fontweight='bold')

# Set the title and legend for the subplots
ax1.legend()
ax2.legend()


# Adjust spacing between subplots
#plt.suptitle("Glucose")
plt.tight_layout()

# Show the plot
plt.show()


In [ ]:
import matplotlib.pyplot as plt

# Plot a histogram of the range values
plt.hist(FVAglu_range, bins=10)
plt.xlabel('Range')
plt.ylabel('Frequency')
plt.title('Distribution of Range Values')
plt.show()